<H1>IMPORT<H1>
    <h2>필요한 라이브러리들 import<h2>

In [2]:
import pandas as pd
import numpy as np
import csv
import requests
import ast
import pickle

<h1>Preprocessing<h1>

In [2]:
csv=pd.read_excel('../data/NDD_DNV_final_list_20210729.xlsx',sheet_name='NDD_DNV_final_20210729',header=1)
#원본데이터를 읽어오기

In [3]:
Neededcolumns=['sampleID','variant','study','inheritance_type','phenotype','Location','Allele','Gene','Feature','Feature_type',
              'Consequence','IMPACT','SYMBOL','SYMBOL_SOURCE','HGNC_ID','BIOTYPE'] #활용할수 도 있는 column들만 추출

In [7]:
csv=csv[Neededcolumns]
csv=csv[csv['inheritance_type']=='denovo']
csv=csv[csv['SYMBOL_SOURCE']=='HGNC']
csv['SYMBOL_SOURCE'].value_counts()
#전체데이터중 denovo이고 HGNC로 표현되 데이터만 수집하기

HGNC    134061
Name: SYMBOL_SOURCE, dtype: int64

In [9]:
csv.to_csv("1차처리.csv", mode='w') #1차 처리후 저장
csv2=csv
sampleinx=0
samples=[]

In [10]:
for idx in csv2.index:   # gene단위로 있는 원본을 person단위로 묶기
    if(csv2.loc[idx,'sampleID'] not in samples):
        samples.append(csv2.loc[idx,'sampleID'])
        sampleinx+=1
        csv2.loc[idx,'personID']=sampleinx
    else: 
        csv2.loc[idx,'personID']=sampleinx


In [24]:
csv2.to_csv("../data/2차처리.csv", mode='w')

<h2>create genedict<h2>
    <h4>kegg api로 genedict key가 될 list를 먼저 호출</h4>

In [ ]:
url2="https://rest.kegg.jp/get/hsa00010"  # api url
response2=requests.get(url=url2)
parsed2=response2.text.split('\n')
print(parsed2)#kegg pathwaylist

In [4]:
genedict={}
def preparegenedict():
    url="https://rest.kegg.jp/list/pathway/hsa"
    response=requests.get(url=url)
    pathwaylist=response.text.split('\n')
    for i in range(len(pathwaylist)-1):
        splitter=pathwaylist[i].split('\t')[0]
        split2=splitter.split(':')
        #print(split2)
        genedict[split2[0]]=[]
    for pathwaynum in genedict.keys():
        url3="https://rest.kegg.jp/get/"+pathwaynum
        response2=requests.get(url=url3)
        parsed2=response2.text.split('\n')
        start=0
        end=0
        for i in range(0,len(parsed2)-1):
            if(parsed2[i][0:4]=='GENE'):
                start=i
            elif(parsed2[i][0:4]=='COMP'):
                end=i
        for i in range(start,end):
            data=parsed2[i]
            s=data.split(';')[0].split("  ")
            gene=s[-1]
            genedict[pathwaynum].append(gene)

In [8]:
with open("genedict.csv", 'w') as file:
    writer = csv.writer(file)
    for k, v in genedict.items():
        writer.writerow([k, v])

In [7]:
genedict2={}

In [ ]:

with open("../data/genedict.csv", 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        new_string = row[1].replace("[", "" )
        new_string = new_string.replace("]", "" )
        new_string = new_string.replace("'", "" )
        new_string = new_string.replace(" ", "" )
        reactomegenedict[row[0]] = new_string.split(',')


for k in reactomegenedict.keys():
        for s in reactomegenedict[k]:
            if s not in reactomeoppodict.keys():
                reactomeoppodict[s]=[]
                reactomeoppodict[s].append(k)
            else:
                reactomeoppodict[s].append(k)



In [80]:

reactomegenedict={}
reactomeoppodict={}
def prereactgenedict():
    with open("../finalreactgenedict.csv", 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            new_string = row[1].replace("[", "" )
            new_string = new_string.replace("]", "" )
            new_string = new_string.replace("'", "" )
            new_string = new_string.replace(" ", "" )
            reactomegenedict[row[0]] = new_string.split(',')
      

    for k in reactomegenedict.keys():
            for s in reactomegenedict[k]:
                if s not in reactomeoppodict.keys():
                    reactomeoppodict[s]=[]
                    reactomeoppodict[s].append(k)
                else:
                    reactomeoppodict[s].append(k)   


    
    

In [3]:
csv2=pd.read_csv('../data/2차처리.csv')
csv2.set_index(keys='Unnamed: 0',inplace=True,drop=True)

In [34]:
unidentified=[]
def preprocess(m):
    cnt=0
    unidentified=[]
    for idx in csv2.index: 
        tmp=0
        if(idx%2000==0):
            print(idx//2000,"%")
        genesymbol=csv2.loc[idx,'SYMBOL']    
        if genesymbol in oppositedict.keys():
            for pathwayname in oppositedict[genesymbol]:
                        tmp+=1
                        if genesymbol in combined.keys():
                                df3.loc[csv2.loc[idx,'personID']-1,pathwayname]+=1.0 * impact(csv2.loc[idx,'IMPACT'])*(combined[genesymbol])*m
            if genesymbol in ppidict2.keys():
                ppimap=ppidict2[genesymbol]
                for j in ppimap.keys():
                        df3.loc[csv2.loc[idx,'personID']-1,j]+= ppimap[j] 
            if tmp==0 and ((csv2.loc[idx,'phenotype']=="ASD") or( csv2.loc[idx,'phenotype']=="control")):
                cnt+=1
                unidentified.append(genesymbol)
        else:
            unidentified.append(genesymbol)
            
    print(cnt)        
        
    

In [8]:
unidentified2=[]
def reactpreprocess():
    cnt=0
    sums=0
    for idx in csv2.index: 
        tmp=0
        if(idx%2000==0):
            print(idx//2000,"%")
        genesymbol=csv2.loc[idx,'SYMBOL']
        if genesymbol in reactomeoppodict.keys():
            for pathwayname in reactomeoppodict[genesymbol]:
                        tmp+=1
                        sums+=1
                        reactdf3.loc[csv2.loc[idx,'personID']-1,pathwayname]+=1.0 * impact(csv2.loc[idx,'IMPACT'])
            if genesymbol in reactppidict2.keys():
                ppimap=reactppidict2[genesymbol]
                for j in ppimap.keys():
                        reactdf3.loc[csv2.loc[idx,'personID']-1,j]+= ppimap[j] 
            if tmp==0 and ((csv2.loc[idx,'phenotype']=="ASD") or( csv2.loc[idx,'phenotype']=="control")):
                cnt+=1
                unidentified2.append(genesymbol)
        else:
            unidentified2.append(genesymbol)       
                
    print(cnt)        
    print(sums)
    print(unidentified2)

In [20]:
reactomeoppodict

{'GFAP': ['R-HSA-9612973', 'R-HSA-9612973'],
 'PINK1': ['R-HSA-9612973', 'R-HSA-9612973'],
 'ATG3': ['R-HSA-9612973'],
 'ATG10': ['R-HSA-9612973'],
 'HSPA8': ['R-HSA-9612973', 'R-HSA-9612973', 'R-HSA-112316'],
 'ATG5': ['R-HSA-9612973'],
 'LAMP2': ['R-HSA-9612973'],
 'SQSTM1': ['R-HSA-9612973'],
 'AMBRA1': ['R-HSA-9612973', 'R-HSA-9612973'],
 'MFN1': ['R-HSA-9612973'],
 'MTERF3': ['R-HSA-9612973'],
 'MFN2': ['R-HSA-9612973'],
 'ATG16L1': ['R-HSA-9612973'],
 'ATG12': ['R-HSA-9612973'],
 'EEF1A1': ['R-HSA-9612973'],
 'VDAC1': ['R-HSA-9612973'],
 'MAD2L1': ['R-HSA-1640170', 'R-HSA-1640170'],
 'CHEK1': ['R-HSA-1640170', 'R-HSA-1640170'],
 'BUB1B': ['R-HSA-1640170'],
 'WEE1': ['R-HSA-1640170', 'R-HSA-1640170', 'R-HSA-109582'],
 'CDC25C': ['R-HSA-1640170', 'R-HSA-1640170', 'R-HSA-1640170'],
 'CHEK2': ['R-HSA-1640170', 'R-HSA-1640170', 'R-HSA-1640170'],
 'GTSE1': ['R-HSA-1640170', 'R-HSA-1640170'],
 'CDC25A': ['R-HSA-1640170'],
 'PKMYT1': ['R-HSA-1640170'],
 'CDC20': ['R-HSA-1640170'],
 'BUB3

In [44]:
len(df3.loc[(df3['phenotype'] >= 0)])

8743

In [17]:
def impact(str):
    if str=="HIGH":
        return 4
    if str=="MODERATE":
        return 3
    if str=="LOW":
        return 2
    if str=="MODIFIER":
        return 1

In [18]:
def impact2(k,n):
    if k=="L":
        return float(n)*0.1
    if k=="M":
        return float(n)*0.3
    if k=="H":
        return float(n)*0.5

In [9]:
autismDB=pd.read_csv('gene-report.csv',header=0)

In [10]:
autismDB=autismDB[autismDB['Report Type']=='Primary']

In [11]:
autismlist=[]
for idx,row in autismDB.iterrows():
    autismlist.append(row['Gene Symbol'])

In [31]:
sfaridict

{'ABAT': 2.0,
 'ABCA13': 2.0,
 'ABCA7': 2.0,
 'ABCE1': 1.0,
 'ABL2': 3.0,
 'ACE': 2.0,
 'ACHE': 2.0,
 'ACTB': 1.0,
 'ACTL6B': 0.0,
 'ACTN4': 2.0,
 'ACY1': 0.0,
 'ADA': 2.0,
 'ADCY3': 2.0,
 'ADCY5': 2.0,
 'ADK': 2.0,
 'ADORA3': 2.0,
 'ADSL': 1.0,
 'ADNP': 1.0,
 'ABCA10': 2.0,
 'ADGRL1': 3.0,
 'AFF2': 1.0,
 'AGAP1': 2.0,
 'AGAP2': 2.0,
 'AGBL4': 2.0,
 'AGMO': 2.0,
 'AGTR2': 2.0,
 'AHDC1': 1.0,
 'AHI1': 0.0,
 'AHNAK': 2.0,
 'AKAP9': 2.0,
 'ALDH1A3': 0.0,
 'ALDH1L1': 3.0,
 'ALDH5A1': 1.0,
 'ALG6': 0.0,
 'AMPD1': 2.0,
 'ANK2': 1.0,
 'AMT': 2.0,
 'AGAP5': 3.0,
 'AGO1': 2.0,
 'AGO2': 2.0,
 'AGO3': 2.0,
 'AGO4': 2.0,
 'ADSS2': 2.0,
 'ANK3': 1.0,
 'ANKRD11': 1.0,
 'ANKRD17': 2.0,
 'ANKS1B': 2.0,
 'ANP32A': 1.0,
 'ANXA1': 2.0,
 'AP1S2': 0.0,
 'AP2S1': 1.0,
 'APBA2': 2.0,
 'APBB1': 2.0,
 'APH1A': 2.0,
 'ARF3': 1.0,
 'ARHGAP32': 2.0,
 'ARHGAP11B': 2.0,
 'ARHGAP5': 2.0,
 'ARHGEF10': 2.0,
 'ARHGEF2': 3.0,
 'ARHGEF9': 1.0,
 'ARID1B': 1.0,
 'ARID2': 2.0,
 'ARNT2': 2.0,
 'ARX': 1.0,
 'ADORA2A': 2.0,
 '

In [12]:
sfaricsv=pd.read_csv('SFARI-Gene_human-gene-scores_04-03-2023release_05-08-2023export.csv',header=0)

In [13]:
sfaricsv=sfaricsv.fillna(0.0)
sfaridict={}
sfarilist=[]
for idx,row in sfaricsv.iterrows():
    sfaridict[row['gene-symbol']]=float(row['gene-score'])
    sfarilist.append(row['gene-symbol'])


In [14]:
combined={}

for i in sfarilist:
    combined[i]=1
for j in autismlist:
    if j in combined.keys():
        combined[j]+=1
    else:
        combined[j]=1
        

In [16]:
cnt=0
for k in combined.keys():
    if combined[k]==1:
        cnt+=1
print(cnt)        

347


In [ ]:
cnt=0
unidentified=[]
for idx in csv2.index: 
    tmp=0
    if(idx%2000==0):
        print(idx//2000,"%")
    genesymbol=csv2.loc[idx,'SYMBOL']
    if genesymbol in reactomeoppodict.keys():
        for pathwayname in reactomeoppodict[genesymbol]:
                    tmp+=1
                    df3.loc[csv2.loc[idx,'personID']-1,pathwayname]+=1.0 * impact(csv2.loc[idx,'IMPACT'])
        if genesymbol in ppidict2.keys():
            ppimap=ppidict2[genesymbol]
            for j in ppimap.keys():
                    df3.loc[csv2.loc[idx,'personID']-1,j]+= ppimap[j] 
    elif csv2.loc[idx,'phenotype']=="control" or csv2.loc[idx,'phenotype']=="ASD":
            cnt+=1
            unidentified.append(genesymbol)
print(cnt)        



In [19]:
def preprocess2(j):
    for idx in csv2.index: 
        phenotype=csv2.loc[idx,'phenotype']
        if phenotype=="ASD":
            df3.loc[csv2.loc[idx,'personID']-1,'phenotype']=1.0 
            continue
        if phenotype=="control":
            df3.loc[csv2.loc[idx,'personID']-1,'phenotype']=0.0
            continue
        else: 
            df3.loc[csv2.loc[idx,'personID']-1,'phenotype']=-1.0

    df4=df3[(df3['phenotype']>=0.0)]
    filename="ppipathway"+str(j)+".csv"
    df4.to_csv(filename, mode='w')

        
    

In [20]:
def reactpreprocess2(j):
    for idx in csv2.index: 
        phenotype=csv2.loc[idx,'phenotype']
        if phenotype=="ASD":
            reactdf3.loc[csv2.loc[idx,'personID']-1,'phenotype']=1.0 
            continue
        if phenotype=="control":
            reactdf3.loc[csv2.loc[idx,'personID']-1,'phenotype']=0.0
            continue
        else: 
            reactdf3.loc[csv2.loc[idx,'personID']-1,'phenotype']=-1.0

    reactdf4=reactdf3[(df3['phenotype']>=0.0)]
    filename="ppipathway"+str(j)+".csv"
    reactdf4.to_csv(filename, mode='w')


In [21]:
def ppi(n):
    for j in ppidict.keys():
        k=ppidict[j]
        ppidict2[j]={}
        for l in k.keys():
            if l in oppositedict.keys():
                m=oppositedict[l]
                for t in m:
                    if t not in ppidict2[j].keys():
                        ppidict2[j][t]=round(impact2(ppidict[j][l],n),1)
                    else:
                        ppidict2[j][t]+=impact2(ppidict[j][l],n)
                        ppidict2[j][t]=round(ppidict2[j][t],1)

In [22]:
def reactppi(n):
    for j in ppidict.keys():
        k=ppidict[j]
        reactppidict2[j]={}
        for l in k.keys():
            if l in reactomeoppodict.keys():
                m=reactomeoppodict[l]
                for t in m:
                    if t not in reactppidict2[j].keys():
                        reactppidict2[j][t]=round(impact2(ppidict[j][l],n),1)
                    else:
                        reactppidict2[j][t]+=impact2(ppidict[j][l],n)
                        reactppidict2[j][t]=round(reactppidict2[j][t],1)

In [23]:

def prepareppi(): 
    f = open("../data/ppi.csv", "r")
    reader = csv.reader(f)
    cnt=0
    for row in reader:
        ppis=row
    

    for j in range(len(ppis)):
        k=ast.literal_eval(ppis[j])
        if k[0] not in ppidict.keys():
            ppidict[k[0]]={}
            ppidict[k[0]][k[1]]=k[2]
        else:
             ppidict[k[0]][k[1]]=k[2]
    ppidict            
    

In [24]:
def readoppositedict(): 
    f = open("../data/oppositedict.csv", "r")
    reader = csv.reader(f)
    first=[]
    cnt=0
    for row in reader:
        first.append(row)
    for i in range(len(first[0])):
        try:
            k=first[0][i].split(';')
            kstr=k[0]
            klist=  ast.literal_eval(first[1][i])    
            oppositedict[kstr]=klist
        except:
            print(kstr)    


In [25]:
ppidict={}
prepareppi()

In [31]:
genedict

{'hsa01100': [],
 'hsa01200': [],
 'hsa01210': [],
 'hsa01212': [],
 'hsa01230': [],
 'hsa01232': [],
 'hsa01250': [],
 'hsa01240': [],
 'hsa00010': ['HK3',
  'HK1',
  'HK2',
  'HKDC1',
  'GCK',
  'GPI',
  'PFKM',
  'PFKP',
  'PFKL',
  'FBP1',
  'FBP2',
  'ALDOC',
  'ALDOA',
  'ALDOB',
  'TPI1',
  'GAPDH',
  'GAPDHS',
  'PGK2',
  'PGK1',
  'PGAM1',
  'PGAM2',
  'PGAM4',
  'ENO3',
  'ENO2',
  'ENO1',
  'ENO4',
  'PKM',
  'PKLR',
  'PDHA2',
  'PDHA1',
  'PDHB',
  'DLAT',
  'DLD',
  'LDHAL6A',
  'LDHAL6B',
  'LDHA',
  'LDHB',
  'LDHC',
  'ADH1A',
  'ADH1B',
  'ADH1C',
  'ADH7',
  'ADH4',
  'ADH5',
  'ADH6',
  'AKR1A1',
  'ALDH2',
  'ALDH3A2',
  'ALDH1B1',
  'ALDH7A1',
  'ALDH9A1',
  'ALDH3B1',
  'ALDH3B2',
  'ALDH3A1',
  'ACSS1',
  'ACSS2',
  'GALM',
  'PGM1',
  'PGM2',
  'G6PC1',
  'G6PC2',
  'G6PC3',
  'ADPGK',
  'BPGM',
  'MINPP1',
  'PCK1',
  'PCK2'],
 'hsa00020': ['CS',
  'ACLY',
  'ACO2',
  'ACO1',
  'IDH1',
  'IDH2',
  'IDH3B',
  'IDH3G',
  'IDH3A',
  'OGDHL',
  'OGDH',
  'DLST',
 

In [32]:
#genedict={}
ppidict={}

#preparegenedict()

prepareppi()

blank=np.zeros((33238,353))
df3=pd.DataFrame(blank)
newcolumn=list(genedict.keys())
newcolumn.append("phenotype")
df3.columns=newcolumn
ppi(1)
preprocess()
preprocess2(1)


ValueError: Length mismatch: Expected axis has 353 elements, new values have 354 elements

In [27]:
oppositedict={}
readoppositedict()

In [45]:
ppidict2={}
blank=np.zeros((33238,354))
df3=pd.DataFrame(blank)
newcolumn=list(genedict.keys())
print(len(newcolumn))
newcolumn.append("phenotype")
df3.columns=newcolumn
ppi(1)
preprocess(3)
preprocess2(1)

353
0 %
1 %
3 %
4 %
9 %
10 %
11 %
12 %
13 %
14 %
16 %
18 %
19 %
21 %
22 %
23 %
24 %
25 %
26 %
27 %
30 %
31 %
34 %
38 %
39 %
43 %
44 %
45 %
46 %
47 %
48 %
49 %
50 %
53 %
54 %
55 %
57 %
60 %
61 %
62 %
63 %
65 %
70 %
72 %
73 %
74 %
75 %
77 %
79 %
81 %
82 %
89 %
90 %
91 %
92 %
93 %
94 %
95 %
96 %
97 %
101 %
102 %
103 %
104 %
0


In [46]:
rowsum=df3.sum(axis=1)
df3['rowsum']=rowsum
falsecontrol=df3[df3['rowsum']==1.0]
falsecontrol=falsecontrol[falsecontrol['phenotype']==1.0]
falseindex=falsecontrol.index
falsedeletedf3=df3.drop(falseindex)

In [47]:
falsedeletedf3=df3.drop(falseindex)

In [48]:
falsedeletedf3=falsedeletedf3[falsedeletedf3['phenotype']>=0.0]
falsedeletedf3=falsedeletedf3[falsedeletedf3.columns.difference(['rowsum'])]  

In [49]:
filename="sfarifalsedeletedppipathwaykeggdf3.3"+".csv"
falsedeletedf3.to_csv(filename, mode='w')

In [83]:
import sys
csv.field_size_limit(sys.maxsize)
reactomegenedict={}
reactomeoppodict={}
ppidict={}

reactppidict2={}
prereactgenedict()
prepareppi()


In [91]:
reactppi(1)
blank=np.zeros((33238,30))
reactdf3=pd.DataFrame(blank)
newcolumn=list(reactomegenedict.keys())
newcolumn.append("phenotype")
reactdf3.columns=newcolumn
reactpreprocess()
reactpreprocess2("reactome")


0 %
1 %
3 %
4 %
9 %
10 %
11 %
12 %
13 %
14 %
16 %
18 %
19 %
21 %
22 %
23 %
24 %
25 %
26 %
27 %
30 %
31 %
34 %
38 %
39 %
43 %
44 %
45 %
46 %
47 %
48 %
49 %
50 %
53 %
54 %
55 %
57 %
60 %
61 %
62 %
63 %
65 %
70 %
72 %
73 %
74 %
75 %
77 %
79 %
81 %
82 %
89 %
90 %
91 %
92 %
93 %
94 %
95 %
96 %
97 %
101 %
102 %
103 %
104 %
0
918437
['DNAH1', 'C12orf55', 'TOM1L1', 'NEK1', 'DDX54', 'PIEZO1', 'SSUH2', 'IFRD1', 'KCNT2', 'NEBL', 'DIP2B', 'ZNF845', 'IGKJ2', 'BRSK2', 'CXXC11', 'PCGF3', 'IGFL2', 'CHD6', 'ADNP2', 'FAM84A', 'TSSK2', 'DNM1P46', 'NPTXR', 'TNS1', 'IGHV5-51', 'PPP1R12C', 'BPIFC', 'SPATA31C1', 'CASKIN2', 'MYOM1', 'TASP1', 'IGHV3-49', 'IGHV1-8', 'C4orf21', 'RALY', 'ZNF536', 'DIS3L2', 'BEND6', 'HTRA2', 'MAATS1', 'TINAG', 'CPXM2', 'ASNSD1', 'KIAA1432', 'USP35', 'ICA1', 'FNDC3A', 'IGHV3-15', 'ANKAR', 'MALRD1', 'FAM53A', 'MAMDC4', 'C15orf43', 'ADNP', 'SHISA2', 'FAM83G', 'UBAC2', 'PTPRE', 'MYO7B', 'TGM3', 'RBP7', 'ZBTB14', 'PCNXL2', 'PLEKHA8P1', 'ACPP', 'MLF1', 'USP40', 'C3orf58', 'CPXM2', 'GPR1

In [113]:
rowsum=reactdf3.sum(axis=1)
reactdf3['rowsum']=rowsum
re_falsecontrol=reactdf3[reactdf3['phenotype']==1.0]
re_falsecontrol

,R-HSA-9612973,R-HSA-1640170,R-HSA-1500931,R-HSA-8953897,R-HSA-4839726,R-HSA-400253,R-HSA-1266738,R-HSA-8963743,R-HSA-1643685,R-HSA-73894,...,R-HSA-1852241,R-HSA-5357801,R-HSA-9609507,R-HSA-1474165,R-HSA-9709957,R-HSA-162582,R-HSA-382551,R-HSA-5653656,phenotype,rowsum
0,3.0,7.0,5.6,1.7,0.0,0.1,133.4,0.0,86.4,2.1,...,0.3,2.0,0.4,0.0,0.3,171.4,2.6,8.6,1.0,9860.9
1,1.3,15.0,2.5,21.0,2.1,0.1,18.2,0.0,156.4,0.0,...,2.0,3.9,0.6,0.9,2.5,183.4,13.4,9.7,1.0,13946.0
2,6.0,40.3,1.4,19.4,1.8,0.5,57.8,0.0,104.4,4.9,...,56.5,5.2,0.0,1.8,7.2,101.2,6.7,20.6,1.0,14930.3
3,0.0,1.5,0.0,3.2,0.3,0.0,1.4,0.2,4.7,2.2,...,0.0,0.8,0.0,0.0,0.0,6.2,0.9,0.4,1.0,1054.9
6,9.1,42.6,27.2,47.2,8.4,3.1,265.6,0.6,637.4,13.8,...,6.3,71.8,1.0,1.6,2.7,1205.6,22.3,34.6,1.0,62594.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33172,23.3,319.4,19.8,103.0,5.1,3.7,136.9,0.3,653.7,141.8,...,27.2,70.7,4.1,5.2,7.4,1043.2,15.1,134.7,1.0,68459.9
33173,31.4,349.8,83.3,198.3,47.7,19.2,425.5,0.0,1014.3,83.1,...,41.3,121.2,5.3,8.1,13.1,2226.8,32.2,103.9,1.0,122125.5
33174,0.0,53.3,0.0,21.4,2.7,1.5,2.7,0.0,58.5,9.4,...,2.7,1.3,0.0,2.1,0.0,46.3,28.8,2.7,1.0,7741.0
33175,11.9,98.0,2.8,79.8,59.2,5.8,122.1,0.0,225.7,63.9,...,4.2,16.2,2.6,3.5,15.0,383.0,13.1,7.4,1.0,30179.3


In [109]:
rowsum=reactdf3.sum(axis=1)
reactdf3['rowsum']=rowsum
reactdf3['phenotype'].value_counts()



-1.0    24495
 1.0     6982
 0.0     1761
Name: phenotype, dtype: int64

In [31]:
len(unidentified2)

58317

In [ ]:
genelist3=list(oppositedict.keys())

In [ ]:
ppidict2={}
createdf()
ppi(1/4)
preprocess()


In [ ]:
with open('genelist.txt', 'w') as fp:
    fp.write('\n'.join(genelist3))

In [55]:
df4.shape

(8743, 354)

In [57]:
for idx in csv2.index: 
        phenotype=csv2.loc[idx,'phenotype']
        if phenotype=="ASD":
            df3.loc[csv2.loc[idx,'personID']-1,'phenotype']=1.0 
            continue
        if phenotype=="control":
            df3.loc[csv2.loc[idx,'personID']-1,'phenotype']=0.0
            continue
        else: 
            df3.loc[csv2.loc[idx,'personID']-1,'phenotype']=-1.0

df4=df3[(df3['phenotype']>=0.0)]
filename="ppipathwaykeggdf4"+".csv"
df4.to_csv(filename, mode='w')

In [27]:
for idx in csv2.index: 
        phenotype=csv2.loc[idx,'phenotype']
        if phenotype=="ASD":
            reactdf3.loc[csv2.loc[idx,'personID']-1,'phenotype']=1.0 
            continue
        if phenotype=="control":
            reactdf3.loc[csv2.loc[idx,'personID']-1,'phenotype']=0.0
            continue
        else: 
            reactdf3.loc[csv2.loc[idx,'personID']-1,'phenotype']=-1.0

reactdf4=reactdf3[(reactdf3['phenotype']>=0.0)]
filename="ppipathwaydf4"+".csv"
reactdf4.to_csv(filename, mode='w')

In [ ]:
pathway2=pd.read_csv('pathway2.csv',)

In [59]:
prepareppi()

In [62]:
sums=0
for i in ppidict.keys():
    sums+=len(ppidict[i])
print(sums)

1778292


In [ ]:
df3['phenotype'].value_counts()

In [ ]:
dfsum.to_csv("sum.csv")

In [ ]:
dfsum.value_counts()

In [ ]:
dfrowsum=df3.sum(axis=1)
dfrowsum=dfrowsum-df3["phenotype"]

In [ ]:
dfrowsum.value_counts()